In [1]:
import torch
import torchvision
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import itertools
from functools import partial

%matplotlib inline

print('cuda:', torch.cuda.is_available())

cuda: True


The code has been referenced from the follwing post :-
http://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

### data loader

In [2]:
transforms = []
transforms += [torchvision.transforms.Resize(224)]
transforms += [torchvision.transforms.RandomCrop(200)]
transforms += [torchvision.transforms.RandomHorizontalFlip()]
transforms += [torchvision.transforms.RandomVerticalFlip()]
transforms += [torchvision.transforms.RandomRotation(degrees=20)]
transforms += [torchvision.transforms.ToTensor()]
transforms += [torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
transforms = torchvision.transforms.Compose(transforms)
trainset = torchvision.datasets.ImageFolder(root='../../data/plant-seed/data/train/', transform=transforms)

In [3]:
transforms = []
transforms += [torchvision.transforms.Resize(224)]
transforms += [torchvision.transforms.CenterCrop(200)]
transforms += [torchvision.transforms.ToTensor()]
transforms += [torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
transforms = torchvision.transforms.Compose(transforms)
evalset = torchvision.datasets.ImageFolder(root='../../data/plant-seed/data/train/', transform=transforms)

In [4]:
transforms = []
transforms += [torchvision.transforms.Resize(224)]
transforms += [torchvision.transforms.CenterCrop(200)]
transforms += [torchvision.transforms.ToTensor()]
transforms += [torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
transforms = torchvision.transforms.Compose(transforms)
validset = torchvision.datasets.ImageFolder(root='../../data/plant-seed/data/valid/', transform=transforms)

### fine-tune model

In [5]:
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 12)
model = model.cuda()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=0.)

In [6]:
def score(model, dataloader):
    
    # initital values
    calc_loss = 0.
    calc_correct = 0.
    calc_count = 0.
    
    for data in dataloader: 
        # next batch
        inputs, labels = data
        inputs = Variable(inputs.cuda())
        labels = Variable(labels.cuda())
        
        # model scores
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        
        # sum the loss and accuracy
        calc_loss += loss.data[0]
        calc_correct += torch.sum(preds == labels.data)
        calc_count += outputs.data.shape[0]     
        
    return round(calc_loss/calc_count, 4), round(calc_correct/calc_count, 4)

In [7]:
def train(trainloader, evalloader, validloader, epochs):
    
    for epoch in range(epochs):
        # initial values
        calc_loss = 0.
        calc_correct = 0.
        calc_count = 0.
        
        # read data
        for data in trainloader: 
            inputs, labels = data
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
            # calculate loss and output
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            # clear gradients
            optimizer.zero_grad()
            # update weights
            loss.backward()
            optimizer.step()
            # track losses
            calc_loss += loss.data[0]
            calc_correct += torch.sum(preds == labels.data)
            calc_count += outputs.data.shape[0]
            
        # evaluate
        print('----------------')
        print('epoch:', epoch)
        print('run loss:', (round(calc_loss/calc_count, 4), round(calc_correct/calc_count, 4)))
        print('train loss:', score(model, evalloader))
        print('valid loss:', score(model, validloader))
        print('----------------')
            
    return model

In [8]:
def adjust_lr(optimizer, value):
    for param_group in optimizer.param_groups:
        param_group['lr'] = value
    return optimizer

### execution

In [9]:
train_loader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=4)
eval_loader  = torch.utils.data.DataLoader(evalset, batch_size=64, shuffle=False, num_workers=4)
valid_loader = torch.utils.data.DataLoader(validset, batch_size=64, shuffle=False, num_workers=4)

In [10]:
model = train(train_loader, eval_loader, valid_loader, 5)

----------------
epoch: 0
run loss: (0.0487, 0.746)
train loss: (0.0547, 0.1535)
valid loss: (0.0399, 0.3453)
----------------
----------------
epoch: 1
run loss: (0.0191, 0.898)
train loss: (0.0605, 0.1593)
valid loss: (0.043, 0.3653)
----------------
----------------
epoch: 2
run loss: (0.015, 0.9185)
train loss: (0.0613, 0.1547)
valid loss: (0.0439, 0.368)
----------------
----------------
epoch: 3
run loss: (0.0122, 0.933)
train loss: (0.064, 0.1713)
valid loss: (0.0445, 0.3787)
----------------
----------------
epoch: 4
run loss: (0.0108, 0.942)
train loss: (0.066, 0.1705)
valid loss: (0.0458, 0.3747)
----------------


In [11]:
optimzer = adjust_lr(optimizer, 0.00005)
model = train(train_loader, eval_loader, valid_loader, 5)

----------------
epoch: 0
run loss: (0.0075, 0.9613)
train loss: (0.065, 0.1742)
valid loss: (0.044, 0.4107)
----------------
----------------
epoch: 1
run loss: (0.0063, 0.9667)
train loss: (0.0635, 0.1797)
valid loss: (0.0432, 0.424)
----------------
----------------
epoch: 2
run loss: (0.0059, 0.9673)
train loss: (0.0635, 0.1722)
valid loss: (0.0418, 0.412)
----------------
----------------
epoch: 3
run loss: (0.0054, 0.97)
train loss: (0.0641, 0.1755)
valid loss: (0.0421, 0.4187)
----------------
----------------
epoch: 4
run loss: (0.0047, 0.9748)
train loss: (0.0658, 0.1782)
valid loss: (0.0437, 0.424)
----------------


In [12]:
optimzer = adjust_lr(optimizer, 0.00001)
model = train(train_loader, eval_loader, valid_loader, 5)

----------------
epoch: 0
run loss: (0.0035, 0.9812)
train loss: (0.065, 0.1755)
valid loss: (0.043, 0.4173)
----------------
----------------
epoch: 1
run loss: (0.0031, 0.9878)
train loss: (0.0646, 0.177)
valid loss: (0.0426, 0.42)
----------------
----------------
epoch: 2
run loss: (0.003, 0.9865)
train loss: (0.0646, 0.1742)
valid loss: (0.0428, 0.424)
----------------
----------------
epoch: 3
run loss: (0.0024, 0.9905)
train loss: (0.0645, 0.179)
valid loss: (0.0426, 0.432)
----------------
----------------
epoch: 4
run loss: (0.0026, 0.9898)
train loss: (0.0643, 0.1782)
valid loss: (0.0425, 0.432)
----------------
